In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
data = pd.read_pickle('../Week1_Strucured_Data_Gen/Dataset_v1_06.pkl')
data.head(4)

,Bird Name,Binomial Name,Synonyms,Old latin name for bird,Kingdom,Subkingdom,Infrakingdom,Phylum,Subphylum,Infraphylum,Superclass,Class,Order,Family,Genus,Species,Conservation status,Habitat,Country,geographic distribution includes,biogeographic realm,population trend,Breeding region,Breeding subregion,visual system,auditory system,hearing range,hearing threshold,locomotion,mineralized skeleton contains,body mass,Wiki_Link,Image,References,Wiki_Description,Ebird_Link,Ebird_Description,Eol_Link,Eol_Description,Dibird_Link
0,abbottsbabbler,Malacocincla abbotti,NaN,NaN,Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Passeriformes,Pellorneidae,Malacocincla,M. abbotti,least concern,forest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_babbler,https://upload.wikimedia.org/wikipedia/commons...,http://ibc.lynxeds.com/species/abbotts-babbler...,Abbott's babbler (Malacocincla abbotti) is a s...,NaN,NaN,NaN,NaN,NaN
1,abbottsbooby,Papasula abbotti,"Abbot's Booby, Spotted-backed Booby",Sula abbotti,Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Suliformes,Sulidae,"PapasulaOlson & Warheit, 1988",P. abbotti,endangered,forest,Christmas Island,NaN,NaN,NaN,Indian ocean,ne,corneal eyes,tympanic middle ear,5000 Hz,20 dB,flight,apatite,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,https://upload.wikimedia.org/wikipedia/commons...,http://www.environment.gov.au/biodiversity/thr...,Abbott's booby (Papasula abbotti) is an endang...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,https://dibird.com/species/abbotts-booby/
2,abbottsstarling,Poeoptera femoralis,NaN,"Pholia femoralis, Pholidauges femoralis, Cinny...",Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Passeriformes,Sturnidae,Poeoptera,P. femoralis,vulnerable,forest,Kenya,Tanzania,NaN,Decreasing,Africa,Kenya and Tanzania,corneal eyes,tympanic middle ear,5000 Hz,10 dB,bipedal movement,apatite,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,https://upload.wikimedia.org/wikipedia/commons...,http://datazone.birdlife.org/species/factsheet...,The Abbott's starling (Poeoptera femoralis) is...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,https://dibird.com/species/abbotts-starling/
3,abbottssunbird,NaN,Souimanga Sunbird (Abbott's),NaN,Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Passeriformes,Nectariniidae,Cinnyris,NaN,not evaluated,NaN,NaN,NaN,NaN,NaN,Africa,Aldabra Island,corneal eyes,tympanic middle ear,5000 Hz,10 dB,flight,apatite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://eol.org/pages/45514925,Cinnyris abbotti (Abbott's Sunbird) is a speci...,https://dibird.com/species/abbotts-sunbird/


In [3]:
find_binomial_manually = data['Binomial Name'].isna()
nobiname_birds = data[find_binomial_manually]
l = list(nobiname_birds['Bird Name'])
# nobiname_birds.head(5)
# l

## EBIRD TRAIL ON MISSING BIRDS

In [4]:
from tkinter import N
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
import time
from datetime import datetime
import logging
from timeit import default_timer as timer  


def get_page_of_bird_ebird(bird_name,driver):
    driver.get('https://ebird.org/explore')
    input_ebird = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[placeholder='Enter species name']")))
    input_ebird.send_keys(bird_name)
    input_ebird.send_keys(Keys.ENTER)
    time.sleep(2)
    input_ebird.send_keys(Keys.ENTER)
    return driver.current_url


# URL = "https://ebird.org/species/grswoo"

def scrape_v_4_ebird(URL):
    if URL != None :
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find_all("div",class_="Breadcrumbs")
        names_main = soup.find_all("span",class_="Heading-main")
        names_sub = soup.find_all("span",class_="Heading-sub")
        udescription=soup.find_all("p",class_="u-stack-sm")
        # print(birds)
        for i,j,k,l in zip(results,names_main,names_sub,udescription):
            a = i.get_text().strip().split('\n')
            b = j.get_text().strip()
            c = k.get_text().strip()
            e = l.get_text()
            d = {'Common Name':[b],'Kingdom':['Animalia'],'Phylum':['Chordata'],'Class':['Aves'],'Order':[a[0]],'Family':[a[1]],'Binomial Name':[c],'Description':[e],'Link':[URL]}
            return pd.DataFrame(d) 
    emp_d = {'Common Name':[b],'Kingdom':pd.notna,'Phylum':pd.notna,'Class':pd.notna,'Order':pd.notna,'Family':pd.notna,'Binomial Name':pd.notna,'Description':pd.notna,'Link':[URL]} 
    
    return pd.DataFrame(emp_d)
    
def ebird(fro,to,driver,birds):
    # types_of_birds_df = pd.read_csv('assets/final_names.csv',index_col='ID')
    # list_of_bird_names = [ types_of_birds_df.loc[ _ ,'Common Name'] for _ in range(1,types_of_birds_df.shape[0]) ]
    list_of_bird_names = l
    for bird_name in list_of_bird_names[fro:to]:
        try:
            birds = pd.concat([birds,scrape_v_4_ebird(get_page_of_bird_ebird(bird_name,driver))] )
            logging.debug('[+] '+ bird_name)
            print('[+] '+ bird_name)
        except Exception:
            logging.error('[-] Unknown Exceptions for Bird ' + bird_name)
    return birds

# if __name__ == '__main__':
# startTime = datetime.now()
# birds = pd.DataFrame(columns=['Common Name','Kingdom','Phylum','Class','Order','Family', 'Binomial Name', 'Link','Description'])
# driver = webdriver.Chrome('./../chromedriver.exe')
# f = int(input('from :'))
# t = int(input('to :'))
# f = 1
# t = len(l)
# ds = 'ebird_'+str(f)+'_'+str(t)
# logging.basicConfig(filename="../assets/rescrape/Re_Log_"+ds+".log",format='%(asctime)s - %(levelname)s - %(message)s',filemode='a')
# birds = ebird(f,t, driver,birds)
# birds.to_pickle("../assets/rescrape/Re_Pick_"+ds+".pkl")
# birds.to_csv("../assets/rescrape/Re_csv_"+ds+".csv")
# logging.info('\n\nTime taken : ',datetime.now() - startTime)
# print('\n\nTime taken : ',datetime.now() - startTime) 
# driver.quit()
    
'''
conda activate indicwiki ; cd Documents\birds ; python Ebird_Scraper_Optim.py
<f>
<t>
'''

'\nconda activate indicwiki ; cd Documents\x08irds ; python Ebird_Scraper_Optim.py\n<f>\n<t>\n'

In [5]:
data.head(4)

,Bird Name,Binomial Name,Synonyms,Old latin name for bird,Kingdom,Subkingdom,Infrakingdom,Phylum,Subphylum,Infraphylum,Superclass,Class,Order,Family,Genus,Species,Conservation status,Habitat,Country,geographic distribution includes,biogeographic realm,population trend,Breeding region,Breeding subregion,visual system,auditory system,hearing range,hearing threshold,locomotion,mineralized skeleton contains,body mass,Wiki_Link,Image,References,Wiki_Description,Ebird_Link,Ebird_Description,Eol_Link,Eol_Description,Dibird_Link
0,abbottsbabbler,Malacocincla abbotti,NaN,NaN,Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Passeriformes,Pellorneidae,Malacocincla,M. abbotti,least concern,forest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_babbler,https://upload.wikimedia.org/wikipedia/commons...,http://ibc.lynxeds.com/species/abbotts-babbler...,Abbott's babbler (Malacocincla abbotti) is a s...,NaN,NaN,NaN,NaN,NaN
1,abbottsbooby,Papasula abbotti,"Abbot's Booby, Spotted-backed Booby",Sula abbotti,Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Suliformes,Sulidae,"PapasulaOlson & Warheit, 1988",P. abbotti,endangered,forest,Christmas Island,NaN,NaN,NaN,Indian ocean,ne,corneal eyes,tympanic middle ear,5000 Hz,20 dB,flight,apatite,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,https://upload.wikimedia.org/wikipedia/commons...,http://www.environment.gov.au/biodiversity/thr...,Abbott's booby (Papasula abbotti) is an endang...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,https://dibird.com/species/abbotts-booby/
2,abbottsstarling,Poeoptera femoralis,NaN,"Pholia femoralis, Pholidauges femoralis, Cinny...",Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Passeriformes,Sturnidae,Poeoptera,P. femoralis,vulnerable,forest,Kenya,Tanzania,NaN,Decreasing,Africa,Kenya and Tanzania,corneal eyes,tympanic middle ear,5000 Hz,10 dB,bipedal movement,apatite,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,https://upload.wikimedia.org/wikipedia/commons...,http://datazone.birdlife.org/species/factsheet...,The Abbott's starling (Poeoptera femoralis) is...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,https://dibird.com/species/abbotts-starling/
3,abbottssunbird,NaN,Souimanga Sunbird (Abbott's),NaN,Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Passeriformes,Nectariniidae,Cinnyris,NaN,not evaluated,NaN,NaN,NaN,NaN,NaN,Africa,Aldabra Island,corneal eyes,tympanic middle ear,5000 Hz,10 dB,flight,apatite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://eol.org/pages/45514925,Cinnyris abbotti (Abbott's Sunbird) is a speci...,https://dibird.com/species/abbotts-sunbird/


In [6]:

data.describe()

,Bird Name,Binomial Name,Synonyms,Old latin name for bird,Kingdom,Subkingdom,Infrakingdom,Phylum,Subphylum,Infraphylum,Superclass,Class,Order,Family,Genus,Species,Conservation status,Habitat,Country,geographic distribution includes,biogeographic realm,population trend,Breeding region,Breeding subregion,visual system,auditory system,hearing range,hearing threshold,locomotion,mineralized skeleton contains,body mass,Wiki_Link,Image,References,Wiki_Description,Ebird_Link,Ebird_Description,Eol_Link,Eol_Description,Dibird_Link
count,11099,10830,9596,8686,11099,11099,11099,11099,11099,11099,11099,11099,11080,11080,10999,10524,10315,8093,7502,5820,4247,5098,10772,10772,5894,5819,2825,4973,6243,5361,3238,10524,10524,10524,10524,5376,5376,6520,6520,10772
unique,11099,10747,9579,8673,1,1,1,1,1,1,1,1,44,257,2381,9439,8,208,180,246,8,4,54,4684,5,2,2,3,9,2,1049,9273,6696,9312,9278,5376,5361,6487,6475,10772
top,abbottsbabbler,Gerygone modesta,"Diuca Finch, Common Diuca-Finch, Common Diucaf...","Pipastes petrosus, Alauda petrosa",Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Passeriformes,Tyrannidae,Zosterops,P. simplex,least concern,forest,Colombia,United States,The Neotropics,Decreasing,South America,widespread,corneal eyes,tympanic middle ear,5000 Hz,10 dB,flight,apatite,14 g,https://en.wikipedia.org/wiki/Conservation_status,https://upload.wikimedia.org/wikipedia/commons...,"['http://www.inbo.be/content/homepage_en.asp',...",The conservation status of a group of organism...,https://ebird.org/species/abbboo2,"Extinct, last reported in 1900.",https://eol.org/pages/1050320,EOL has data for one attribute:,https://dibird.com/species/abbotts-booby/
freq,1,2,2,2,11099,11099,11099,11099,11099,11099,11099,11099,6612,438,102,7,7854,4288,701,873,1340,2448,2553,594,5886,5445,2534,3172,3330,5360,49,996,996,566,995,1,10,2,10,1


In [7]:
data.drop_duplicates(inplace = True)

In [8]:
data.describe()

,Bird Name,Binomial Name,Synonyms,Old latin name for bird,Kingdom,Subkingdom,Infrakingdom,Phylum,Subphylum,Infraphylum,Superclass,Class,Order,Family,Genus,Species,Conservation status,Habitat,Country,geographic distribution includes,biogeographic realm,population trend,Breeding region,Breeding subregion,visual system,auditory system,hearing range,hearing threshold,locomotion,mineralized skeleton contains,body mass,Wiki_Link,Image,References,Wiki_Description,Ebird_Link,Ebird_Description,Eol_Link,Eol_Description,Dibird_Link
count,11099,10830,9596,8686,11099,11099,11099,11099,11099,11099,11099,11099,11080,11080,10999,10524,10315,8093,7502,5820,4247,5098,10772,10772,5894,5819,2825,4973,6243,5361,3238,10524,10524,10524,10524,5376,5376,6520,6520,10772
unique,11099,10747,9579,8673,1,1,1,1,1,1,1,1,44,257,2381,9439,8,208,180,246,8,4,54,4684,5,2,2,3,9,2,1049,9273,6696,9312,9278,5376,5361,6487,6475,10772
top,abbottsbabbler,Gerygone modesta,"Diuca Finch, Common Diuca-Finch, Common Diucaf...","Pipastes petrosus, Alauda petrosa",Animalia,Bilateria,Deuterostomia,Chordata,Vertebrata,Gnathostomata,Tetrapoda,Aves,Passeriformes,Tyrannidae,Zosterops,P. simplex,least concern,forest,Colombia,United States,The Neotropics,Decreasing,South America,widespread,corneal eyes,tympanic middle ear,5000 Hz,10 dB,flight,apatite,14 g,https://en.wikipedia.org/wiki/Conservation_status,https://upload.wikimedia.org/wikipedia/commons...,"['http://www.inbo.be/content/homepage_en.asp',...",The conservation status of a group of organism...,https://ebird.org/species/abbboo2,"Extinct, last reported in 1900.",https://eol.org/pages/1050320,EOL has data for one attribute:,https://dibird.com/species/abbotts-booby/
freq,1,2,2,2,11099,11099,11099,11099,11099,11099,11099,11099,6612,438,102,7,7854,4288,701,873,1340,2448,2553,594,5886,5445,2534,3172,3330,5360,49,996,996,566,995,1,10,2,10,1


In [9]:
# %pip install -U dataprep

In [10]:
# from dataprep.eda import plot
# import numpy as np
# plot(data)